### 1. 전처리 및 데이터 클래스 정의

#### 1) Library Import

In [1]:
import glob
import random
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import numpy as np
import math
import itertools
import datetime
import time
from torchvision.utils import save_image, make_grid
from torchvision import datasets
from torch.autograd import Variable

#### 2) 흑백 이미지를 RGB 이미지로 바꾸는 함수

`to_rgb` : PIL의 Image 모듈을 이용해 인자로 받은 흑백 이미지를 RGB로 바꿔서 반환

In [2]:
def to_rgb(image) :
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image

#### 3) 사용자 정의 데이터셋 클래스 정의

- 정의한 데이터셋으로 Loader를 이용해서 배치 사이즈만큼 이미지를 불러올 수 있다.

In [3]:
class ImageDataset(Dataset) :
    def __init__(self, root, transforms_=None, unaligned=False, mode="train") :
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned
        if mode == "train" :
            self.files_A = sorted(glob.glob(os.path.join(root, "trainA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "trianB") + "/*.*"))
        else :
            self.fiels_A = sorted(glob.glob(os.path.join(root, "testA") + "/*.*"))
            self.files_B = sorted(glob.glob(os.path.join(root, "testB") + "/*.*"))
            
    def __getitem__(self, index) :
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        
        
        # unaligned 변수로 학습할 쌍을 무작위로 고를지 고정시킬지 ?
        if self.unaligned :
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B)-1)])
        else :
            image_B = Image.open(self.files_B[index % len(self.files_B)])
            
        
        if image_A.mode != "RGB" :
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB" :
            image_B = to_rgb(image_B)
            
        # 불러온 PIL 이미지를 인자로 받은 transform 함수를 적용하여 torch의 tensor 자료형으로 바꾼다. 
        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A" : item_A, "B" : item_B}
    
    
    def __len__(self) :
        return max(len(self.files_A), len(self.files_B))
            

-----------------------------------------------------------------------------------------------------------

### 2. Generator 구현

- Generator는 이미지를 입력받아 다른 스타일의 이미지를 생성하는 역할

#### 1) 가중치 초기화 함수

- 토치에서 제공하는 layer의 종류에 따라 가중치 초기화를 다르게 하고자 아래의 함수를 사용
- __class__로 layer의 이름을 얻은 후에 각 종류에 맞게 가중치를 초기화한다.

In [4]:
def weights_init_normal(m) :
    classname = m.__class__.__name__
    if classname.find("Conv") != -1 :
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None :
            torch.nn.init.constant_(m.bias.data, 0.0)
            
    elif classname.find("BatchNorm2d") != -1 :
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

#### 2) Residual block 구현

- Generator를 구현하기 위해서는 내부에 들어갈 layer인 Residual Block을 구현해야 한다.
- Residual Block 
    - 이전 layer와 현재 layer의 출력값을 더해서 Forward
    - 모델이 깊어짐에 따라 생기는 기울기 소실 문제 해결
    - 더하기 연산으로는 기울기가 작아지지 않고 그대로 Back Propagation이 일어나기 때문

In [7]:
class ResidualBlock(nn.Module) :
    def __init__(self, in_features) :
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )
    def forward(self, x) :
        return x + self.block(x)

- `ReflectionPad` : 점대칭 방식으로 가장 가까운 픽셀로부터 값을 복사
    - Zero Padding처럼 값 지정이 아닌 더욱 자연스러운 이미지 생성을 위해 사용
- `Instance Normalization` : 데이터 개별로 정규화 진행
    - 정규화 : 데이터 값을 범위를 비슷하게 조정
    - 배치 정규화는 데이터의 배치 단위로 평균과 분산을 구하여 학습의 안정성을 높이지만, Instance Normalization은 이미지에 특화된 정규화 과정으로 이미지를 개별로 정규화한다.

#### 3) Generator 구현

- 입력 이미지를 down-sampling한 후, 
- 여러개의 Residual Block을 통과시킨 후에
- Up-sampling하는 것으로 스타일을 변환하는 Generator 구현

In [8]:
class GeneratorResNet(nn.Module) :
    def __init__(self, input_shape, num_residual_blocks) :
        super(GeneratorResNet, self).__init__()
        channels = input_shape[0]
        
        # 초기 Convolutional block 선언
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features
        
        
        # Downsampling을 2번 진행한다. (stride=2이므로 이미지의 크기가 반씩 줄어든다.)
        for _ in range(2) :
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
            
        
        # num_residual_blocks만큼 residual block 만들기
        for _ in range(num_residual_blocks) :
            model += [ResidualBlock(out_features)]
            
            
        # nn.Upsample을 2번 진행하여 다시 이미지의 크기를 2배씩 늘린다.
        for _ in range(2) :
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
            
            
        # 출력 layer를 선언한다. (출력 이미지의 크기 = 입력 이미지의 크기)
        # 마지막 층의 활성 함수는 nn.Tanh() 사용
        model += [nn.ReflectionPad2d(channels), 
                  nn.Conv2d(out_features, channels, 7),
                  nn.Tanh()]
        self.model = nn.Sequential(*model)
        
    
    def forward(self, x) :
        return self.model(x)

-----------------------------------------------------------------------------------------------------------

### 3. Discriminator 구현

- 입력받은 이미지가 실제 이미지인지 생성된 이미지인지를 분류하는 역할
- (PatchGAN의 Discriminator 기반) 
    - 정사각형 사이즈의 이미지 패치 영역에 대하여 생성된 이미지가 가짜인지 진짜인지 판단
    - 이미지 영역을 분할하지 않는다면, Generator는 우리가 학습하려는 스타일의 변환이 아닌 엉뚱한 특징으로 Discriminator를 속인다.
    - 이를 방지하기 위해 각각의 패치 영역을 따로 판단하여 원하는 스타일의 변환을 학습할 수 있다.
    - 또한 작은 이미지 패치에 대하여 연산을 수행하므로 파라미터의 개수가 작아지고 속도가 더 빠르다.

#### 1) Discriminator 구현

In [17]:
class Discriminator(nn.Module) :
    def __init__(self, input_shape) :
        super(Discriminator, self).__init__()
        channels, height, width = input_shape
        
        
        # Discriminator의 출력 크기를 정한다.
        # PatchGAN의 Discriminator는 출력이 0또는 1의 값이 아니라 입력 이미지의 1/16인 이진화된 Feature Map
        # 만약 이미지의 크기가 256x256이라면 Discriminator의 출력 이미지는 16x16
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)
        
        
        # discriminator block은 stride=2로 점점 Downsampling하면서 출력 이미지의 크기를 줄인다. 
        def discriminator_block(in_filters, out_filters, normalize=True) :
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize :
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        
        # 이미지의 크기가 256x256일 때, discriminator_block을 4번 통과하면 한번 통과할 때마다 크기가 반으로 줄어들므로,
        # 출력 이미지의 크기는 16x16
        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )
                
        
        
    def forward(self, img) :
        return self.model(img)

-----------------------------------------------------------------------------------------------------------

### 4. 모델 학습

#### 1) 하이퍼파라미터 지정

- `dataset_name` : 학습 및 테스트 데이터가 들어 있는 폴더 혹은 폴더의 이름
- `channels` : 이미지의 채널(흑백 : 1, RGB 이미지 : 3)
- `img_height`, `img_width` : 이미지의 가로, 세로의 크기
- `n_residual_blocks` : Generator에서의 Residual Block의 개수
- `lr` : 모델에 대한 Learning Rate
- `b1`, `b2`  : Adam Optimizer에 대한 HyperParameter
- `lambda_cyc`, `lambda_id` : Cycle-consistency Loss와 Identity Loss에 대한 람다 값
    - lambda_id가 클수록 본래의 색감을 유지하려는 성질


In [15]:
dataset_name = "selfie2anime"
channels = 3
img_height = 256
img_width = 256
n_residual_blocks = 9
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_epochs = 200
init_epoch = 0
decay_epoch = 100
lambda_cyc = 10.0
lambda_id = 5.0
n_cpu = 8
batch_size = 1
sample_interval = 100
checkpoint_interval = 5

#### 2) 샘플 이미지와 모델 가중치를 저장할 폴더 생성

- 학습하는 동안 생성할 샘플 이미지와 학습시킬 모델을 저장할 폴더 생성
- `exist_ok = True`로 지정하면 같은 이름의 폴더가 있어도 오류가 나지 않는다.

In [11]:
os.makedirs("images/%s" % dataset_name, exist_ok=True)
os.makedirs("saved_models/%s" % dataset_name, exist_ok=True)

#### 3) 손실 함수 정의하기

In [12]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

#### 4) 모델 객체 선언하기

- 스타일을 변환하기 위한 Generator, Discriminator 선언
    - G_AB : A에서 B로 변환 
    - G_BA : B에서 A로 변환
    - D_A,D_B : 생성한 스타일 A, B가 진짜인지 가짜인지 판별할 네트워크

In [18]:
input_shape = (channels, img_height, img_width)

# Initialize generator and discriminator
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)

In [49]:
!pip install torchinfo

In [50]:
from torchinfo import summary
batch_size = 1
summary(GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9), input_size = (batch_size, 3, 256, 256))

Layer (type:depth-idx)                        Output Shape              Param #
GeneratorResNet                               [1, 3, 256, 256]          --
├─Sequential: 1-1                             [1, 3, 256, 256]          --
│    └─ReflectionPad2d: 2-1                   [1, 3, 262, 262]          --
│    └─Conv2d: 2-2                            [1, 64, 256, 256]         9,472
│    └─InstanceNorm2d: 2-3                    [1, 64, 256, 256]         --
│    └─ReLU: 2-4                              [1, 64, 256, 256]         --
│    └─Conv2d: 2-5                            [1, 128, 128, 128]        73,856
│    └─InstanceNorm2d: 2-6                    [1, 128, 128, 128]        --
│    └─ReLU: 2-7                              [1, 128, 128, 128]        --
│    └─Conv2d: 2-8                            [1, 256, 64, 64]          295,168
│    └─InstanceNorm2d: 2-9                    [1, 256, 64, 64]          --
│    └─ReLU: 2-10                             [1, 256, 64, 64]          --
│    └─R

In [54]:
from torchinfo import summary
batch_size = 1
summary(GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9), input_size=((batch_size, 3, 256, 256)))

Layer (type:depth-idx)                        Output Shape              Param #
GeneratorResNet                               [1, 3, 256, 256]          --
├─Sequential: 1-1                             [1, 3, 256, 256]          --
│    └─ReflectionPad2d: 2-1                   [1, 3, 262, 262]          --
│    └─Conv2d: 2-2                            [1, 64, 256, 256]         9,472
│    └─InstanceNorm2d: 2-3                    [1, 64, 256, 256]         --
│    └─ReLU: 2-4                              [1, 64, 256, 256]         --
│    └─Conv2d: 2-5                            [1, 128, 128, 128]        73,856
│    └─InstanceNorm2d: 2-6                    [1, 128, 128, 128]        --
│    └─ReLU: 2-7                              [1, 128, 128, 128]        --
│    └─Conv2d: 2-8                            [1, 256, 64, 64]          295,168
│    └─InstanceNorm2d: 2-9                    [1, 256, 64, 64]          --
│    └─ReLU: 2-10                             [1, 256, 64, 64]          --
│    └─R

In [51]:
from torchsummaryX import summary
summary(GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9), torch.zeros((1, 3, 256, 256)))

TypeError: summary() missing 1 required positional argument: 'x'

In [41]:
import torchsummary
torchsummary.summary(GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9), input_size=(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1          [-1, 3, 262, 262]               0
            Conv2d-2         [-1, 64, 256, 256]           9,472
    InstanceNorm2d-3         [-1, 64, 256, 256]               0
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
    InstanceNorm2d-6        [-1, 128, 128, 128]               0
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8          [-1, 256, 64, 64]         295,168
    InstanceNorm2d-9          [-1, 256, 64, 64]               0
             ReLU-10          [-1, 256, 64, 64]               0
  ReflectionPad2d-11          [-1, 256, 66, 66]               0
           Conv2d-12          [-1, 256, 64, 64]         590,080
   InstanceNorm2d-13          [-1, 256, 64, 64]               0
             ReLU-14          [-1, 256,

In [47]:
import pytorch_model_summary
import torch
print(pytorch_model_summary.summary(GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9), 
                                    torch.zeros(1, 3, 256, 256), show_input=True))

-----------------------------------------------------------------------------
         Layer (type)            Input Shape         Param #     Tr. Param #
    ReflectionPad2d-1       [1, 3, 256, 256]               0               0
             Conv2d-2       [1, 3, 262, 262]           9,472           9,472
     InstanceNorm2d-3      [1, 64, 256, 256]               0               0
               ReLU-4      [1, 64, 256, 256]               0               0
             Conv2d-5      [1, 64, 256, 256]          73,856          73,856
     InstanceNorm2d-6     [1, 128, 128, 128]               0               0
               ReLU-7     [1, 128, 128, 128]               0               0
             Conv2d-8     [1, 128, 128, 128]         295,168         295,168
     InstanceNorm2d-9       [1, 256, 64, 64]               0               0
              ReLU-10       [1, 256, 64, 64]               0               0
     ResidualBlock-11       [1, 256, 64, 64]       1,180,160       1,180,16

In [19]:
print(G_AB)

GeneratorResNet(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        

#### 5) GPU에 로드하기

- `torch.cuda.is_available()` : GPU 연산이 가능한지 확인

In [58]:
cuda = torch.cuda.is_available()
if cuda :
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

#### 6) 가중치 초기화

- 앞서 선언한 weights_init_normal 함수로 Generator와 Discriminator의 가중치를 초기화한다.
- `apply` 함수로 각 네트워크에 있는 모든 layer에 가중치 초기화를 적용할 수 있다.

In [60]:
G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d((1, 0, 1, 0))
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

#### 7) 옵티마이저 정의

- Generator와 Discriminator는 Adam 사용
- `itertools.chain` : Optimizer가 여러 모델의 파라미터를 하나의 모델을 다루는 것처럼 동작한다.

In [61]:
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()),
    lr = lr,
    betas = (b1, b2)
)

optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))

#### 8) 학습 스케쥴러

- LambdaLR 클래스 정의
    - Learning Rate를 Decay할 Epoch을 정할 수 있다.

In [65]:
class LambdaLR :
    def __init__(self, n_epochs, offset, decay_start_epoch) :
        assert (n_epochs - decay_start_epoch) > 0 , "Decay must start before the training session ends !"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch
        
    def step(self, epoch) :
        return 1.0 - max(0, epoch+self.offset-self.decay_start_epoch) / (self.n_epochs-self.decay_start_epoch)

- Learning Rate Scheduler 정의

In [67]:
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step)

lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step)

lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step)

- Tensor 연산에 사용할 Tensor 자료형 정의

In [66]:
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

#### 9) ReplayBuffer

- Torch 변수가 requires_grad = True로 지정되어 있다면 매 연산마다 Gradient를 저장하므로 CycleGAN의 학습을 위해서는 ReplayBuffer 클래스를 통해 이미지를 따로 저장해야 한다.

In [68]:
class ReplayBuffer :
    def __init__(self, max_size = 50) :
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []
        
    def push_and_pop(self, data) :
        to_return = []
        for element in data.data :
            element = torch.unsqueeze(element, 0)
            if len(self.data) > self.max_size :
                self.data.append(element)
                to_return.append(elment)
            else :
                if random.uniform(0, 1) > 0.5 :
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else :
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [70]:
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

#### 10) transform 정의

- Dataset 클래스로 이미지를 불러오는 방식에 대하여 정의

In [71]:
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

C:\Users\ATIV\AppData\Local\Temp\ipykernel_11356\3287249881.py:2: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
C:\Users\ATIV\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


#### 11) DataLoader 정의

- 학습 혹은 평가 중 이미지 데이터를 불러올 DataLoader 정의
- 앞서 정의한 Image Dataset 클래스로 selfie2anime 데이터 폴더로부터 transform_을 적용한 이미지를 배치 사이즈 만큼 불러온다.
- num_workers로 cpu 유틸리티를 설정할 수 있다.
- shuffle 변수를 True 혹은 False로 설정해 이미지를 무작위로 혹은 순차적으로 불러올 수 있다.

In [74]:
# Training data loader
dataloader = DataLoader(
    ImageDataset("./datasets/%s" % dataset_name, transforms_ = transforms_, unaligned = True),
    batch_size = batch_size,
    shuffle = True,
    num_workers = n_cpu,
)

# Test data loader
val_dataloader = DataLoader(
    ImageDataset("./datasets/%s" % dataset_name, transforms_ = transforms_, unaligned = True, mode="test"),
    batch_size = 5,
    shuffle=True,
    num_workers=1,
)

ValueError: num_samples should be a positive integer value, but got num_samples=0

#### 12) 생성한 샘플 이미지를 시각화하여 저장하는 함수

- val_dataloader의 배치 사이즈가 5이므로 sample_images는 `make_grid` 함수를 통해 5개씩 샘플을 생성하여 images 폴더에 저장한다.

In [75]:
def sample_images(batches_done) :
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    real_B = Variable(imgs["B"].type(Tensor))
    fake_A = G_BA(real_B)
    
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    save_image(image_grid, "images/%s/%s.png" % (dataset_name, batches_done), normalize=False)

#### 13) 모델 학습 파이프라인 및 최종 학습 과정

- (1) Set model input
    - dataloader에서 실제 사진 이미지(A)와 애니메이션 이미지(B)를 배치 사이즈만큼 불러온다.
- (2) Adversarial ground truths
    - Discriminator의 레이블 값을 만든다.
    - valid와 fake 변수를 만든다.
    - PatchGAN에 따라 valid는 Discriminator의 출력 크기만큼 전부 1로 채워지고, fake는 Discriminator의 출력 크기만큼 전부 0으로 채워진다.
- (3) Train Generators
    - Generator G_AB, G_BA를 학습 모드로 전환
    - optimizer_G를 zero_grad()
- (4) Identity Loss
    - 색감, 형태 등을 유지하기 위한 Identity loss 계산
    - G_BA에 실제 이미지 A를 입력한 후, 이를 real_A와 비교하여 L1Loss 계산
    - G_AB도 마찬가지
- (5) GAN Loss
    - 이미지 real_A로부터 스타일이 변환된 가짜 이미지 fake_B를 생성
    - D_B : 생성된 fake_B가 진짜인지 가짜인지 분류한다.
    - D_B를 속이도록 G_AB의 GAN loss를 계산한다.
    - real_B도 마찬가지 과정을 거쳐서 G_BA의 GAN loss 계산
- (6) Cycle Loss
    - G_BA로 가짜 이미지 fake_B에서 새로운 가짜 이미지 recov_A를 생성
    - 이를 다시 원래 실제 이미지 real_A와 비교하여 L1 loss 계산
- (7) Total Loss
    - 앞서 계산한 loss를 총합하여 Generator의 전체 손실 함수를 계산하고 Generator의 가중치 업데이트
    
- (8) Train Discriminator A
    - D_A의 가중치를 업데이트하는 과정
    - 실제 이미지 real_A는 valid로 분류하고, fake_A는 fake로 분류
    - 학습을 위해 optimizer_D_A를 zero_grad() 한다.
- (9) Real Loss
    - D_A가 진짜라고 판별한 경우, 실제 이미지 real_A의 MSE 손실함수를 계산한다.
- (10) Fake Loss (on batch of previously generated samples)
    - D_A가 가짜라고 판별한 경우, 가짜 이미지 fake_A_의 MSE 손실함수를 계산한다.
    - fake_A의 가중치는 업데이트 가능한 상태이므로 detach() 함수로 값만 복사해온다.
- (11) Total Loss
    - (9), (10)에서 계산한 손실함수를 모두 합한 후 D_A의 가중치를 업데이트한다. 
    
- (12) Train Discriminator B
    - D_B의 가중치를 업데이트하는 과정도 D_A의 가중치를 업데이트 하는 것과 동일하다.
    - D_B는 실제 이미지 real_B를 valid로 분류하고, fake_B는 fake로 분류한다.
- (13) Real Loss
    - D_B가 진짜라고 판별한 경우, 실제 이미지 real_B의 MSE 손실함수를 계산한다. 
- (14) Fake loss (on batch of previously generated samples)
    - D_B가 가짜라고 판별한 경우, 가짜 이미지 fake_B의 MSE 손실함수를 계산한다.
- (15) Total Loss
    - (13), (14)에서 계산한 손실함수를 모두 합한 후 D_B의 가중치를 업데이트

- (16) Determine approximate time left
    - Epoch와 batch size로 남은 시간을 출력하기 위한 코드
- (17) Print log
    - Epoch와 batch size로 남은 시간을 출력하기 위한 코드
- (18) If at sample interval save image
    - 특정 epoch 간격마다 샘플로 생성한 이미지를 저장
- (19) Update learning rates
    - Generator와 Discriminator의 Learning Rate 스케쥴러를 업데이트
- (20) Save model checkpoints
    - 모델 가중치 저장

In [78]:
prev_time = time.time()
for epoch in range(init_epoch, n_epochs) :
    for i, batch in enumerate(dataloader) :
        
        # (1) Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))
        
        # (2) Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))),
                         requires_grad = False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))),
                        requires_grad = False)
        
        # (3) Train Generators
        G_AB.train()
        G_BA.train()
        optimizer_G.zero_grad()
        
        # (4) Identity Loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)
        loss_identity = (loss_id_A + loss_id_B) / 2
        
        # (5) GAN Loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)
        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
        
        # (6) Cycle Loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)
        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2
        
        # (7) Total Loss
        loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity
        logg_G.backward()
        optimizer_G.step()
        
        # (8) Train Discriminator A
        optimizer_D_A.zero_grad()
        
            # (9) Real Loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        
            # (10) Fake Loss (on batch of previously generated samples)
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        
            # (11) Total Loss
        loss_D_A = (loss_real + loss_fake) / 2
        loss_D_A.backward()
        optimizer_D_A.step()
        
        
        # (12) Train Discriminator B
        optimizer_D_B.zero_grad()
        
            # (13) Real Loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        
            # (14) Fake Loss (on batch of previously generated samples)
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        
            # (15) Total Loss
        loss_D_B = (loss_real + loss_fake) / 2
        loss_D_B.backward()
        optimizer_D_B.step()
        
        loss_D = (loss_D_A + loss_D_B) / 2
        
        # (16) Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds = batches_left * (time.time()-prev_time))
        prev_time = time.time()
        
        # (17) Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss:%f, adv: %f, cycle: %f, identity: %f] ETA: %s" % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )
        # (18) If at sample interval save image
        if batches_done % sample_interval == 0 :
            sample_images(batches_done)
            
    # (19) Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()
    
    # (20) Save model checkpoints
    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (dataset_name, epoch))
        torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (dataset_name, epoch))
        torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (dataset_name, epoch))
        torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (dataset_name, epoch))

TypeError: object of type 'type' has no len()